<a href="https://colab.research.google.com/github/LES4975/Album_Art_Generator_Project/blob/main/Album_Art_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

첫 번째 셀은 인터페이스만 표시하는 데모 코드입니다. 시연할 때 실행하지 않아도 됩니다.

여기까지가 데모 코드입니다.
본 코드는 아래부터 이어집니다.

In [ ]:
# main.py
# 메인 실행 스크립트 - 모든 초기화 작업 포함

"""
Album Art Generator - 메인 실행 파일

사용법:
1. Google Colab에서 이 파일을 실행
2. 모든 초기화가 자동으로 진행됩니다
3. Gradio 앱이 자동으로 시작됩니다
"""

import os
import sys
from pathlib import Path
import subprocess

def install_libraries():
    """필요한 라이브러리 설치"""
    print("📦 라이브러리 설치 중...")

    # 라이브러리 목록
    libraries = [
        "gradio",
        "diffusers",
        "transformers",
        "accelerate",
        '"numpy==1.26.4"',  # numpy 버전 고정
        "essentia-tensorflow --no-deps",  # Essentia 설치
        "pyyaml",
        "six"
    ]

    for lib in libraries:
        try:
            print(f"  설치 중: {lib}")
            subprocess.run(f"pip install {lib} --force-reinstall", shell=True, check=True, capture_output=True)
        except subprocess.CalledProcessError as e:
            print(f"  ⚠️ {lib} 설치 실패: {e}")

    print("✅ 라이브러리 설치 완료")

def setup_environment():
    """환경 설정"""
    print("🔧 환경 설정 중...")

    # 1. 라이브러리 설치
    install_libraries()

    # 2. Google Drive 마운트
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("✅ Google Drive 마운트 완료")
    except ImportError:
        print("⚠️ Google Colab 환경이 아닙니다")
        return False
    except Exception as e:
        print(f"❌ Google Drive 마운트 실패: {e}")
        return False

    print("✅ 환경 설정 완료")
    return True


In [ ]:
def setup_model_files():
    """모델 파일 설정 및 심볼링크 생성"""
    print("🔗 모델 파일 설정 중...")

    # 기본 경로
    drive_path = "/content/drive/MyDrive/album_art_generator_project"

    # 모델 파일 경로들
    model_files = {
        "discogs_model": f"{drive_path}/discogs-effnet-bs64-1.pb",
        "mood_model": f"{drive_path}/mtg_jamendo_moodtheme-discogs-effnet-1.pb",
        "discogs_json": f"{drive_path}/discogs-effnet-bs64-1.json",
        "mood_json": f"{drive_path}/mtg_jamendo_moodtheme-discogs-effnet-1.json"
    }

    # 파일 존재 확인
    missing_files = []
    for name, file_path in model_files.items():
        if Path(file_path).exists():
            print(f"✅ {name}: 존재함")
        else:
            print(f"❌ {name}: 없음")
            missing_files.append(file_path)

    if missing_files:
        print(f"\n⚠️ 누락된 모델 파일 {len(missing_files)}개:")
        for file_path in missing_files:
            print(f"   - {file_path}")
        return False

    # 작업 디렉토리에 심볼링크 생성
    try:
        symlinks = [
            (model_files["discogs_model"], "discogs-effnet-bs64-1.pb"),
            (model_files["mood_model"], "mtg_jamendo_moodtheme-discogs-effnet-1.pb"),
            (model_files["discogs_json"], "discogs-effnet-bs64-1.json"),
            (model_files["mood_json"], "mtg_jamendo_moodtheme-discogs-effnet-1.json")
        ]

        for source, target in symlinks:
            if not Path(target).exists():
                os.symlink(source, target)
                print(f"🔗 심볼링크 생성: {target}")
            else:
                print(f"✅ 심볼링크 존재: {target}")

        print("✅ 모델 파일 설정 완료")
        return True

    except Exception as e:
        print(f"❌ 심볼링크 생성 실패: {e}")
        return False

In [ ]:
def check_files():
    """필수 파일 존재 확인"""
    print("📁 파일 확인 중...")

    # 기본 경로
    drive_path = "/content/drive/MyDrive/album_art_generator_project"

    # 필수 파일들
    required_files = [
        f"{drive_path}/discogs-effnet-bs64-1.pb",
        f"{drive_path}/mtg_jamendo_moodtheme-discogs-effnet-1.pb",
        f"{drive_path}/discogs-effnet-bs64-1.json",
        f"{drive_path}/mtg_jamendo_moodtheme-discogs-effnet-1.json",
        f"{drive_path}/music_classification_colab.py"
    ]

    missing_files = []
    for file_path in required_files:
        if Path(file_path).exists():
            print(f"✅ {Path(file_path).name}")
        else:
            print(f"❌ {Path(file_path).name}")
            missing_files.append(file_path)

    if missing_files:
        print(f"\n⚠️ 누락된 파일 {len(missing_files)}개:")
        for file_path in missing_files:
            print(f"   - {file_path}")
        print("\n해결 방법:")
        print("1. Google Drive에 필요한 파일들을 업로드하세요")
        print("2. 파일 경로가 올바른지 확인하세요")
        return False

    print("✅ 모든 필수 파일 확인 완료")
    return True



In [ ]:
def initialize_music_classifier():
    """음악 분류기 초기화"""
    print("🎵 음악 분류기 초기화 중...")

    try:
        # 경로 추가
        sys.path.append('/content/drive/MyDrive/album_art_generator_project')

        # Essentia import 테스트
        try:
            import essentia
            print(f"✅ Essentia 버전: {essentia.__version__}")
        except Exception as e:
            print(f"❌ Essentia 로드 실패: {e}")
            print("Runtime을 재시작해야 할 수 있습니다")
            return None

        # 음악 분류기 생성
        from music_classification_colab import ColabMusicClassifier
        classifier = ColabMusicClassifier()

        print("✅ 음악 분류기 초기화 완료")
        return classifier

    except Exception as e:
        print(f"❌ 음악 분류기 초기화 실패: {e}")
        return None

def test_system():
    """시스템 테스트"""
    print("🧪 시스템 테스트 중...")

    try:
        # numpy 버전 확인
        import numpy as np
        print(f"📊 numpy 버전: {np.__version__}")

        # Essentia 테스트
        import essentia
        print(f"🎵 Essentia 버전: {essentia.__version__}")

        # torch 확인 (GPU)
        import torch
        print(f"🔥 PyTorch GPU 사용 가능: {torch.cuda.is_available()}")

        print("✅ 시스템 테스트 완료")
        return True

    except Exception as e:
        print(f"❌ 시스템 테스트 실패: {e}")
        return False


In [ ]:
def main():
    """메인 실행 함수"""
    print("🎵 Album Art Generator 시작")
    print("=" * 50)

    # 1. 환경 설정
    if not setup_environment():
        print("❌ 환경 설정 실패로 종료합니다")
        return

    # 2. 파일 확인
    if not check_files():
        print("❌ 필수 파일이 누락되어 실행을 중단합니다")
        return

    # 3. 모델 파일 설정
    if not setup_model_files():
        print("❌ 모델 파일 설정 실패로 종료합니다")
        return

    # 4. 시스템 테스트
    if not test_system():
        print("❌ 시스템 테스트 실패. Runtime 재시작 후 다시 시도하세요")
        return

    # 5. 음악 분류기 초기화
    classifier = initialize_music_classifier()
    if classifier is None:
        print("❌ 음악 분류기 초기화 실패로 종료합니다")
        return

    # 6. 앱 실행
    try:
        print("🚀 Gradio 앱 시작...")

        # 모듈 임포트
        from gradio_app import create_and_launch_app

        # 앱 실행 (초기화된 분류기 전달)
        print("🎯 앱 시작...")
        create_and_launch_app(
            music_classifier_path="/content/drive/MyDrive/album_art_generator_project",
            classifier_instance=classifier  # 초기화된 분류기 전달
        )

    except ImportError as e:
        print(f"❌ 모듈 임포트 실패: {e}")
        print("album_art_core.py와 gradio_app.py 파일이 현재 디렉토리에 있는지 확인하세요")
    except Exception as e:
        print(f"❌ 앱 실행 실패: {e}")

def restart_runtime_warning():
    """Runtime 재시작 안내"""
    print("\n" + "="*60)
    print("⚠️  RUNTIME 재시작이 필요할 수 있습니다")
    print("="*60)
    print("만약 Essentia 관련 오류가 발생한다면:")
    print("1. Runtime > Restart runtime 클릭")
    print("2. 이 스크립트를 다시 실행하세요")
    print("="*60)


In [ ]:
if __name__ == "__main__":
    restart_runtime_warning()

    # 메인 실행
    main()


⚠️  RUNTIME 재시작이 필요할 수 있습니다
만약 Essentia 관련 오류가 발생한다면:
1. Runtime > Restart runtime 클릭
2. 이 스크립트를 다시 실행하세요
🎵 Album Art Generator 시작
🔧 환경 설정 중...
📦 라이브러리 설치 중...
  설치 중: gradio
  설치 중: diffusers
  설치 중: transformers
  설치 중: accelerate
  설치 중: "numpy==1.26.4"
  설치 중: essentia-tensorflow --no-deps
  설치 중: pyyaml
  설치 중: six
✅ 라이브러리 설치 완료
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive 마운트 완료
✅ 환경 설정 완료
📁 파일 확인 중...
✅ discogs-effnet-bs64-1.pb
✅ mtg_jamendo_moodtheme-discogs-effnet-1.pb
✅ discogs-effnet-bs64-1.json
✅ mtg_jamendo_moodtheme-discogs-effnet-1.json
✅ music_classification_colab.py
✅ 모든 필수 파일 확인 완료
🔗 모델 파일 설정 중...
✅ discogs_model: 존재함
✅ mood_model: 존재함
✅ discogs_json: 존재함
✅ mood_json: 존재함
✅ 심볼링크 존재: discogs-effnet-bs64-1.pb
✅ 심볼링크 존재: mtg_jamendo_moodtheme-discogs-effnet-1.pb
✅ 심볼링크 존재: discogs-effnet-bs64-1.json
✅ 심볼링크 존재: mtg_jamendo_moodtheme-discogs-effnet-1.json
✅ 모델 파일 설정 완료
🧪 시스템